In [1]:
# 필요한 라이브러리 import
import pandas as pd
import numpy as np
import geopandas as gpd
import json

In [2]:
# csv 파일로 저장된 raw 데이터 df로 불러옴 
df = pd.read_csv('output/서울시_상권분석서비스(영역-상권).csv', encoding='utf-8')
df2 = pd.read_csv('output/cd_summary_view.csv', encoding='utf-8')

In [3]:
# raw 데이터 확인
df.head(10)

,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,엑스좌표_값,와이좌표_값,자치구_코드,자치구_코드_명,행정동_코드,행정동_코드_명,영역_면적
0,A,골목상권,3110008,배화여자대학교(박노수미술관),197093,453418,11110,종로구,11110515,청운효자동,149264
1,A,골목상권,3110009,자하문터널,196991,455057,11110,종로구,11110550,부암동,178306
2,A,골목상권,3110010,평창동서측,197064,456643,11110,종로구,11110560,평창동,369415
3,A,골목상권,3110017,정독도서관,198581,453781,11110,종로구,11110600,가회동,83855
4,A,골목상권,3110018,중앙고등학교,198883,453690,11110,종로구,11110600,가회동,166872
5,A,골목상권,3110019,창덕궁,199222,452926,11110,종로구,11110615,"종로1,2,3,4가동",40999
6,A,골목상권,3110020,서울국제고등학교,199598,454302,11110,종로구,11110650,혜화동,133692
7,A,골목상권,3110001,이북5도청사,196262,456660,11110,종로구,11110560,평창동,108529
8,A,골목상권,3110002,독립문역 1번,196220,452912,11110,종로구,11110570,무악동,31531
9,A,골목상권,3110003,세검정초등학교,196389,455998,11110,종로구,11110550,부암동,191570


In [4]:
# 바꾸고자 하는 x, y좌표가 기록된 칼럼 지정
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['엑스좌표_값'], df['와이좌표_값']))

# 현재 좌표계 설정
gdf.crs = 'epsg:5181'

# 좌표계 변환
gdf_4162 = gdf.to_crs('epsg:4004')

# 변환 좌표 속성 생성 
gdf['경도'] = gdf_4162['geometry'].x
gdf['위도'] = gdf_4162['geometry'].y

In [5]:
# 변환된 좌표 확인
gdf.head(5)

,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,엑스좌표_값,와이좌표_값,자치구_코드,자치구_코드_명,행정동_코드,행정동_코드_명,영역_면적,geometry,경도,위도
0,A,골목상권,3110008,배화여자대학교(박노수미술관),197093,453418,11110,종로구,11110515,청운효자동,149264,POINT (197093.000 453418.000),126.967090,37.580309
1,A,골목상권,3110009,자하문터널,196991,455057,11110,종로구,11110550,부암동,178306,POINT (196991.000 455057.000),126.965928,37.595076
2,A,골목상권,3110010,평창동서측,197064,456643,11110,종로구,11110560,평창동,369415,POINT (197064.000 456643.000),126.966748,37.609366
3,A,골목상권,3110017,정독도서관,198581,453781,11110,종로구,11110600,가회동,83855,POINT (198581.000 453781.000),126.983935,37.583584
4,A,골목상권,3110018,중앙고등학교,198883,453690,11110,종로구,11110600,가회동,166872,POINT (198883.000 453690.000),126.987354,37.582764


In [6]:
# 지도에 표시할 view 테이블 데이터 확인
df2.head(5)

,cd_cd,sum_cm_sa,sum_total_ns,ratio_cm_sa_ns,max_total_ns_st_nm,max_cm_sa_st_nm
0,3001491,9.128453e+10,1678,5.440079e+07,일반의류,양식음식점
1,3001492,4.011166e+11,8273,4.848502e+07,일반의류,한식음식점
2,3001493,1.184935e+11,11774,1.006399e+07,일반의류,일반의류
3,3001494,2.564394e+11,4953,5.177456e+07,조명용품,한식음식점
4,3001495,1.685818e+11,2431,6.934668e+07,일반의류,한식음식점


In [7]:
# json 형식으로 변환
df2_list = df2.to_dict(orient='records')
df2_list

[{'cd_cd': 3001491,
  'sum_cm_sa': 91284533308.0,
  'sum_total_ns': 1678,
  'ratio_cm_sa_ns': 54400794.58164482,
  'max_total_ns_st_nm': '일반의류',
  'max_cm_sa_st_nm': '양식음식점'},
 {'cd_cd': 3001492,
  'sum_cm_sa': 401116610220.0,
  'sum_total_ns': 8273,
  'ratio_cm_sa_ns': 48485024.805995405,
  'max_total_ns_st_nm': '일반의류',
  'max_cm_sa_st_nm': '한식음식점'},
 {'cd_cd': 3001493,
  'sum_cm_sa': 118493466767.0,
  'sum_total_ns': 11774,
  'ratio_cm_sa_ns': 10063994.119840326,
  'max_total_ns_st_nm': '일반의류',
  'max_cm_sa_st_nm': '일반의류'},
 {'cd_cd': 3001494,
  'sum_cm_sa': 256439388280.0,
  'sum_total_ns': 4953,
  'ratio_cm_sa_ns': 51774558.50595599,
  'max_total_ns_st_nm': '조명용품',
  'max_cm_sa_st_nm': '한식음식점'},
 {'cd_cd': 3001495,
  'sum_cm_sa': 168581775543.0,
  'sum_total_ns': 2431,
  'ratio_cm_sa_ns': 69346678.54504319,
  'max_total_ns_st_nm': '일반의류',
  'max_cm_sa_st_nm': '한식음식점'},
 {'cd_cd': 3001496,
  'sum_cm_sa': 207146333466.0,
  'sum_total_ns': 787,
  'ratio_cm_sa_ns': 263210080.6429479,
 

In [8]:
# 상권 코드로 검색해서 정보를 불러오도록 key 값을 추가
df2_dict = {
    df2_list[i]['cd_cd'] : df2_list[i] for i in range(len(df2_list))
}
df2_dict

{3001491: {'cd_cd': 3001491,
  'sum_cm_sa': 91284533308.0,
  'sum_total_ns': 1678,
  'ratio_cm_sa_ns': 54400794.58164482,
  'max_total_ns_st_nm': '일반의류',
  'max_cm_sa_st_nm': '양식음식점'},
 3001492: {'cd_cd': 3001492,
  'sum_cm_sa': 401116610220.0,
  'sum_total_ns': 8273,
  'ratio_cm_sa_ns': 48485024.805995405,
  'max_total_ns_st_nm': '일반의류',
  'max_cm_sa_st_nm': '한식음식점'},
 3001493: {'cd_cd': 3001493,
  'sum_cm_sa': 118493466767.0,
  'sum_total_ns': 11774,
  'ratio_cm_sa_ns': 10063994.119840326,
  'max_total_ns_st_nm': '일반의류',
  'max_cm_sa_st_nm': '일반의류'},
 3001494: {'cd_cd': 3001494,
  'sum_cm_sa': 256439388280.0,
  'sum_total_ns': 4953,
  'ratio_cm_sa_ns': 51774558.50595599,
  'max_total_ns_st_nm': '조명용품',
  'max_cm_sa_st_nm': '한식음식점'},
 3001495: {'cd_cd': 3001495,
  'sum_cm_sa': 168581775543.0,
  'sum_total_ns': 2431,
  'ratio_cm_sa_ns': 69346678.54504319,
  'max_total_ns_st_nm': '일반의류',
  'max_cm_sa_st_nm': '한식음식점'},
 3001496: {'cd_cd': 3001496,
  'sum_cm_sa': 207146333466.0,
  'sum_to

In [9]:
# 해당 상권 코드로 생성된 view 테이블 데이터가 없을 경우를 위해 빈 데이터 추가
for i in list(map(int, np.array(gdf['상권_코드']))):
    if i not in df2_dict:
        df2_dict[i] = {
            'cd_cd': i,
            'sum_cm_sa': 0,
            'sum_total_ns': 0,
            'ratio_cm_sa_ns': 0,
            'max_total_ns_st_nm': '없음',
            'max_cm_sa_st_nm': '없음'
        }

In [10]:
'''
각각의 칼럼 데이터를 numpy array로 만들어서 문자열 데이터는 그대로 리스트에 저장,
숫자 데이터는 각각의 데이터 타입으로 변환시켜 리스트에 저장 (numpy array로 생성된 숫자 배열은 int64이기 때문에 변환 필요),
zip으로 묶은 뒤 반복문으로 양식에 입력

'''
geopoint = {
    "type": "FeatureCollection",
	"name": "haengpoint",
	"features": [
        {
			"type": "Feature",
			"geometry": {
				"type": "Point",
				"coordinates": [lon, lat]
			},
			"properties": {
				"cd_div_cd": sgc,
				"cd_div_nm": sgcn,
				"cd_cd": sc,
                "cd_nm": scn,
                "br_cd": jc,
                "br_nm": jcn,
                "ab_cd": hc,
                "ab_nm": hcn,
                "sum_cm_sa": df2_dict[sc]['sum_cm_sa'],
                "sum_total_ns": df2_dict[sc]['sum_total_ns'],
                "ratio_cm_sa_ns": df2_dict[sc]['ratio_cm_sa_ns'],
                "max_total_ns_st_nm": df2_dict[sc]['max_total_ns_st_nm'],
                "max_cm_sa_st_nm": df2_dict[sc]['max_cm_sa_st_nm']
			}
		} for sgc, sgcn, sc, scn, jc, jcn, hc, hcn, lat, lon in zip(list(np.array(gdf['상권_구분_코드'])),
                                                              list(np.array(gdf['상권_구분_코드_명'])),
                                                              list(map(int, np.array(gdf['상권_코드']))),
                                                              list(np.array(gdf['상권_코드_명'])),
                                                              list(map(int, np.array(gdf['자치구_코드']))),
                                                              list(np.array(gdf['자치구_코드_명'])),
                                                              list(map(int, np.array(gdf['행정동_코드']))),
                                                              list(np.array(gdf['행정동_코드_명'])),
                                                              list(map(float, np.array(gdf['위도']))),
                                                              list(map(float, np.array(gdf['경도']))))
    ]
}

In [11]:
geopoint_list = [
    {
        'cd_div_cd': geopoint['features'][i]['properties']['cd_div_cd'],
        'cd_div_nm': geopoint['features'][i]['properties']['cd_div_nm'],
        'cd_cd': geopoint['features'][i]['properties']['cd_cd'],
        'cd_nm': geopoint['features'][i]['properties']['cd_nm'],
        'br_cd': geopoint['features'][i]['properties']['br_cd'],
        'br_nm': geopoint['features'][i]['properties']['br_nm'],
        'ab_cd': geopoint['features'][i]['properties']['ab_cd'],
        'ab_nm': geopoint['features'][i]['properties']['ab_nm'],
        'abn_nm': ' '.join(['서울특별시', geopoint['features'][i]['properties']['br_nm'], geopoint['features'][i]['properties']['ab_nm']]),
        'sum_cm_sa': geopoint['features'][i]['properties']['sum_cm_sa'],
        'sum_total_ns': geopoint['features'][i]['properties']['sum_total_ns'],
        'ratio_cm_sa_ns': geopoint['features'][i]['properties']['ratio_cm_sa_ns'],
        'max_total_ns_st_nm': geopoint['features'][i]['properties']['max_total_ns_st_nm'],
        'max_cm_sa_st_nm': geopoint['features'][i]['properties']['max_cm_sa_st_nm'],
        'coordinates': geopoint['features'][i]['geometry']['coordinates']
    } for i in range(len(geopoint['features']))
]

In [15]:
# 생성된 geopoint json 형식 확인
geopoint['features'][0]

{'type': 'Feature',
 'geometry': {'type': 'Point',
  'coordinates': [126.9670895683088, 37.58030943835253]},
 'properties': {'cd_div_cd': 'A',
  'cd_div_nm': '골목상권',
  'cd_cd': 3110008,
  'cd_nm': '배화여자대학교(박노수미술관)',
  'br_cd': 11110,
  'br_nm': '종로구',
  'ab_cd': 11110515,
  'ab_nm': '청운효자동',
  'sum_cm_sa': 5713974840.0,
  'sum_total_ns': 255,
  'ratio_cm_sa_ns': 22407744.470588237,
  'max_total_ns_st_nm': '한식음식점',
  'max_cm_sa_st_nm': '커피-음료'}}

In [16]:
geopoint_list[0]

{'cd_div_cd': 'A',
 'cd_div_nm': '골목상권',
 'cd_cd': 3110008,
 'cd_nm': '배화여자대학교(박노수미술관)',
 'br_cd': 11110,
 'br_nm': '종로구',
 'ab_cd': 11110515,
 'ab_nm': '청운효자동',
 'abn_nm': '서울특별시 종로구 청운효자동',
 'sum_cm_sa': 5713974840.0,
 'sum_total_ns': 255,
 'ratio_cm_sa_ns': 22407744.470588237,
 'max_total_ns_st_nm': '한식음식점',
 'max_cm_sa_st_nm': '커피-음료',
 'coordinates': [126.9670895683088, 37.58030943835253]}

In [17]:
# json 파일로 output 폴더에 저장
with open('output/geopoint.json', 'w') as f:
    json.dump(geopoint, f, indent=4, ensure_ascii=False)

with open('output/geopoint_list.json', 'w') as f:
    json.dump(geopoint_list, f, indent=4, ensure_ascii=False)

In [18]:
# 데이터베이스에 적재를 위해 좌표계 변환 후 불필요한 칼럼 제거
gdf = gdf.drop(columns=['geometry','엑스좌표_값','와이좌표_값'])
gdf

,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,자치구_코드,자치구_코드_명,행정동_코드,행정동_코드_명,영역_면적,경도,위도
0,A,골목상권,3110008,배화여자대학교(박노수미술관),11110,종로구,11110515,청운효자동,149264,126.967090,37.580309
1,A,골목상권,3110009,자하문터널,11110,종로구,11110550,부암동,178306,126.965928,37.595076
2,A,골목상권,3110010,평창동서측,11110,종로구,11110560,평창동,369415,126.966748,37.609366
3,A,골목상권,3110017,정독도서관,11110,종로구,11110600,가회동,83855,126.983935,37.583584
4,A,골목상권,3110018,중앙고등학교,11110,종로구,11110600,가회동,166872,126.987354,37.582764
...,...,...,...,...,...,...,...,...,...,...,...
1645,U,관광특구,3001492,명동 남대문 북창동 다동 무교동 관광특구,11140,중구,11140520,소공동,983618,126.981856,37.564059
1646,U,관광특구,3001493,동대문패션타운 관광특구,11140,중구,11140590,광희동,606058,127.011274,37.567321
1647,U,관광특구,3001494,"종로,청계 관광특구",11110,종로구,11110615,"종로1,2,3,4가동",653127,126.997691,37.570007
1648,U,관광특구,3001495,잠실 관광특구,11710,송파구,11710562,방이2동,2462734,127.115241,37.516431


In [ ]:
# csv 파일로 저장
gdf.to_csv('raw/서울시_상권분석서비스(영역-상권)_epsg_4162.csv', encoding='utf-8')